# Forensics - C. Memory Analysis 1 - Process Investigator

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ azure 4.0.0,  azure-cli-profile 2.1.4<br>
__OS Supported:__<br>
    -  Linux
    -  Windows
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides sample code to use custom script extension to run Processor Investigator (memory dump) on Linux and Windows.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Initialize Azure Resource Management Clients 
2. Manage Blob Storage
3. Create VM Extension
4. Read Memory Dump Data
5. Delete VM Extension

## 1. Initialize Azure Resource Management Clients

In [ ]:
# User Input and Save to Environmental store
import os
import ipywidgets as widgets
from IPython.display import display
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.storage import StorageManagementClient
import SentinelAzure
from SentinelWidgets import *
from azure.storage.blob import BlockBlobService, PageBlobService, AppendBlobService
from azure.storage.blob.models import BlobBlock, ContainerPermissions, ContentSettings

# User Input and Save to Environmental store
# Set to True if you want to reset the environmental values
reset_environment_valiables = False
env_dir = %env
env_list = ['tenant_domain', 'subscription_id', 'resource_group_for_vm', 'resource_group_for_storage']
envs =  {env_list[0]:'', env_list[1]:'', env_list[2]:'', env_list[3]:''}
envs = WidgetViewHelper.set_env(reset_environment_valiables, env_dir, envs)
tenant_domain = envs[env_list[0]]
subscription_id = envs[env_list[1]]
resource_group_for_vm = envs[env_list[2]]
resource_group_for_storage = envs[env_list[3]]
# Counting visitors
from SentinelLog import *
logger = LogHelper(reset_environment_valiables, env_dir, envs, 'PI')
logger.count_page_view()

In [ ]:
# SKIP THIS IF YOU HAVE RUN SSO USING DEVICE in Part 0-B
# Blank cell for select an auth method
# Copy Code here

In [ ]:
# SKIP THIS IF YOU HAVE RUN SSO USING DEVICE in Part 0-B
# Blank cell for authentication
# Copy Code here

In [ ]:
# RUN THIS IF YOU HAVE RUN SSO DEVICE in Part 0-B
auth = SentinelAzure.azure_aad_helper.AADHelper()
from azure.common.client_factory import get_client_from_cli_profile
# please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
#!az login --tenant $tenant_domain
compute_client = get_client_from_cli_profile(ComputeManagementClient, subscription_id = subscription_id)
storage_client = get_client_from_cli_profile(StorageManagementClient, subscription_id = subscription_id)
compute = SentinelAzure.azure_compute_helper.ComputeHelper(compute_client, resource_group_for_vm)
storage = SentinelAzure.azure_storage_helper.StorageHelper(storage_client)

## 2. Manage Blob Storage

1. Storage Account for storing results

In [ ]:
# Select creating a new account or using existing one
selected_method = WidgetViewHelper.select_account_creation()
display(selected_method)

In [ ]:
storage_account = None
if selected_method.value == 'Creating new account':
    # usert input storage account name
    storage_account_name = WidgetViewHelper.check_storage_account_name_availability(storage)
    if storage_account_name is None:
        print('Name is already used, please re-run this cell and provide a different name')
    else:
        storage_key = WidgetViewHelper.create_storage_account_and_get_key(storage, storage_account_name, resource_group_for_storage)
else:
    selected_storage_account_name = WidgetViewHelper.select_storage_account(storage, resource_group_for_storage)
    display(selected_storage_account_name)

In [ ]:
# Get storage account for selected existing account
if selected_method.value == 'Using exist account':
    storage_account_name = selected_storage_account_name.value
    storage_key = storage.get_storage_account_key(storage_account_name, resource_group_for_storage)

3. Blob Storage Container

In [ ]:
# Select creating a new blob container or using existing one
selected_blob_method = WidgetViewHelper.select_blob_container_creation()
display(selected_blob_method)

In [ ]:
# Get blob container
if selected_blob_method.value == 'Creating new container':
    blob_container_name = input('blob_container_name:')
    storage.initialize_block_blob_service(storage_account_name, storage_key, blob_container_name)
    blob_container = storage.create_blob_container()
else:
    selected_blob_container = WidgetViewHelper.select_blob_container(storage, resource_group_for_storage, storage_account_name)
    display(selected_blob_container)

In [ ]:
if selected_blob_method.value == 'Using exist container':
    storage.initialize_block_blob_service(storage_account_name, storage_key, selected_blob_container.value)

In [ ]:
# Select targeting operating system
selected_os_type = WidgetViewHelper.select_os()
display(selected_os_type)

In [ ]:
# generating upload path for results
sas_expiration_in_days = 10
upload_container_path = WidgetViewHelper.generate_upload_container_path(storage, selected_os_type.value, sas_expiration_in_days)

## 3. Create VM Extension

1. Initialization

In [ ]:
# preparing vm_extension_properties
command_to_execute, file_list = WidgetViewHelper.get_vm_extension_properties(selected_os_type.value, upload_container_path)
if selected_os_type.value == 'Windows':
    vm_extension_properties = SentinelAzure.WindowsVMExtensionProperties(command_to_execute, file_list)
elif selected_os_type.value == 'Linux':
    vm_extension_properties = SentinelAzure.LinuxVMExtensionProperties(command_to_execute, file_list)

In [ ]:
# Select a VM
selected_vm = WidgetViewHelper.select_vm(compute)
display(selected_vm)

In [ ]:
# Collecting values
vm_extension_name = 'sentinelmemoryinvestigator'
vm_name = selected_vm.value
vm_location = compute.get_vm(vm_name).location

2. Create VM Extension

In [ ]:
vm_extension = compute.initialize_vm_extension(vm_extension_properties, vm_location)
create_vm_async = compute.create_vm_extension_async(vm_name, vm_extension_name, vm_extension)
print(create_vm_async.provisioning_state)

## 4. Read Memory Dump Data

In [ ]:
from pandas.io.json import json_normalize
returned_json = compute.get_uploaded_result(upload_container_path)

1. Machine Information

In [ ]:
df_stats = json_normalize(returned_json['machine'])
display(df_stats)

2. Procrss List

In [ ]:
df_process_list = json_normalize(returned_json['processList'])
display(df_process_list)

3. Conclusion

In [ ]:
df_summary = json_normalize(returned_json['scanSummary'])
display(df_summary)

## 5. Delete VM Extension as a final step

In [ ]:
del_result = compute.delete_vm_extension_async(vm_name, vm_extension_name)

## Feedback

In [ ]:
#1. Do you think this notebook helpful?
logger.ask_is_helpful()

In [ ]:
#2. Appreciate your feedback
logger.ask_feedback()